National Instruments Multifunction DAQ example
=======================================

This was written for/tested with [National Instruments USB-6363 DAQ](https://www.ni.com/en-us/support/model.usb-6363.html), but the [nidaqmx](https://nidaqmx-python.readthedocs.io/en/latest/) API is pretty general, so I expect it will work with other devices
with minimal changes. The driver currently only supports analog inputs and outputs, no digital I/O.

In [ ]:
import nidaqmx
from qcodes_contrib_drivers.drivers.NationalInstruments.DAQ import DAQAnalogInputs, DAQAnalogOutputs

## `DAQAnalogInputs`

Acquire 1 million points simultaneously on `num_ai_channels` with a sample rate of 1 MHz, averaging the acquired data down to 100 thousand points.

In [ ]:
devname = 'Dev0' # can be found with NI-MAX
sample_rate_per_channel = 1e6 # Hz
num_ai_channels = 5
ai_channels = {'meas{}'.format(i): i for i in range(num_ai_channels)}
num_samples_raw = 1e6
num_samples_averaged = 1e5

In [ ]:
with nidaqmx.Task('ai_task') as ai_task:
    daq_ai = DAQAnalogInputs(
        'daq_ai',
        devname,
        sample_rate_per_channel,
        ai_channels,
        ai_task,
        samples_to_read=num_samples_raw,
        target_points=num_samples_averaged,
    )
    ai_task.start()
    result = daq_ai.voltage() # result.shape == (num_ai_channels, num_samples_averaged)
    ai_task.wait_until_done()
    ai_task.stop()

For an example of synchronously writing and acquiring data on many channels, see the [`scanning-squid` docs](https://scanning-squid.readthedocs.io/en/latest/), in particular [`microscope.susceptometer.Susceptometer.scan_surface`](https://scanning-squid.readthedocs.io/en/latest/_modules/microscope/susceptometer.html#SusceptometerMicroscope.scan_surface) and [`scanner.Scanner.scan_line`](https://scanning-squid.readthedocs.io/en/latest/_modules/scanner.html#Scanner.scan_line).

## `DAQAnalogOutputs`

`DAQAnalogOutputs` functions as a simple multichannel DC DAC.

In [ ]:
devname = 'Dev0' # can be found with NI-MAX
ao_channels = {str(i): i for i in range(5)}
daq_ao = DAQAnalogOutputs('daq_ao', devname, ao_channels)
daq_ao.voltage_0(5)
daq_ao.voltage_4(2.2)